# [Sec 0] submit a avg score csv
data set in trainging.csv
- id: ad identifier
- click: 0/1 for non-click/click
- hour: format is YYMMDDHH, so 14091123 means 23:00 on Sept. 11, 2014 UTC. *
- C1 – anonymized categorical variable *
- banner_pos
- site_id *
- site_domain *
- site_category *
- app_id *
- app_domain *
- app_category  *
- device_id
- device_ip *
- device_model
- device_type *
- device_conn_type *
- C14-C21 – anonymized categorical variables

In [1]:
import pandas as pd

In [2]:
sampleSubmission = pd.read_csv("data/sampleSubmission")

In [3]:
sampleSubmission.click = 0.16980562476404604
sampleSubmission.head()

,id,click
0,10000174058809263569,0.169806
1,10000182526920855428,0.169806
2,10000554139829213984,0.169806
3,10001094637809798845,0.169806
4,10001377041558670745,0.169806


In [4]:
# all avg-score can reach rank (1314/1604) 81 %
sampleSubmission.to_csv("avg_submission.csv",index=False)

# [Sec1] Train a xgb model with minimum features

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import random

/home/stream/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# training data is to large, sample ~0.5N
N= 40428967 
s = int(N/2) #desired sample size
skip = sorted(random.sample(range(1,N+1),N-s)) 
X_train = pd.read_csv("data/train", skiprows=skip)


In [8]:
# feat
from sklearn.feature_extraction import FeatureHasher
hasher = FeatureHasher(n_features=10,
            non_negative=True,
            input_type='string')
tmp_site_id = hasher.transform(X_train.site_id.tolist()).toarray()
for i in range(10):
    X_train['Site_id'+str(i)]=tmp_site_id[:,i]

In [9]:
Site_list = ['Site_id'+str(i) for i in range(10)]
X_train_trial = X_train[['banner_pos','C1','device_type','device_conn_type','C14','C15','C16','C17','C18',\
                         'C19','C20','C21']+Site_list]
y_target = X_train.click.values

In [10]:
msk = np.random.rand(len(X_train_trial)) < 0.8
dtrain = xgb.DMatrix(X_train_trial[msk], y_target[msk])
dtest = xgb.DMatrix(X_train_trial[~msk], y_target[~msk])

In [ ]:
param = {'max_depth':8, 'eta':0.5, 'silent':1, 'objective':'binary:logistic' ,'subsample':0.3}
param['nthread'] = 6
param['eval_metric'] = 'logloss'
evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 8000
bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=300)

[0]	eval-logloss:0.511968	train-logloss:0.511717
Multiple eval metrics have been passed: 'train-logloss' will be used for early stopping.

Will train until train-logloss hasn't improved in 300 rounds.
[1]	eval-logloss:0.45266	train-logloss:0.452279
[2]	eval-logloss:0.42931	train-logloss:0.428822
[3]	eval-logloss:0.420055	train-logloss:0.419517
[4]	eval-logloss:0.415532	train-logloss:0.41496
[5]	eval-logloss:0.413285	train-logloss:0.412684
[6]	eval-logloss:0.411675	train-logloss:0.411065
[7]	eval-logloss:0.410772	train-logloss:0.41014
[8]	eval-logloss:0.409882	train-logloss:0.409239
[9]	eval-logloss:0.409373	train-logloss:0.408714
[10]	eval-logloss:0.409052	train-logloss:0.408383
[11]	eval-logloss:0.408686	train-logloss:0.408013
[12]	eval-logloss:0.408359	train-logloss:0.407673
[13]	eval-logloss:0.407953	train-logloss:0.407256
[14]	eval-logloss:0.407703	train-logloss:0.407008
[15]	eval-logloss:0.407551	train-logloss:0.40686
[16]	eval-logloss:0.407365	train-logloss:0.406671
[17]	eval-log